In [1]:
import pandas as pd
import datetime
import numpy as np
from pandas_datareader import data as web
from matplotlib import pyplot as pp

In [8]:
zscore = lambda x:(x -x.mean())/x.std() # zscore: normalization of log returns

In [9]:
start = datetime.datetime(2012,1,1)
end = datetime.datetime(2016,1,1)

In [10]:
df = web.DataReader("GOOGL", "google", start, end)

In [11]:
def _load_data(data, n_prev):  
    """
    data should be pd.DataFrame()
    """

    docX, docY = [], []
    for i in range(len(data)-n_prev):
        zscore(data.iloc[i:i+n_prev+1])
        docX.append(zscore(data.iloc[i:i+n_prev]).as_matrix())
        docY.append(zscore(data.iloc[i:i+n_prev+1]).Close[30])
    alsX = np.array(docX)
    alsY = np.array(docY)
    
    return alsX, alsY



In [12]:
def train_test_split(df, test_size=0.1):  
    """
    This just splits data to training and testing parts
    """
    ntrn = int(round(len(df) * (1 - test_size)))

    X_train, y_train = _load_data(df.iloc[0:ntrn],30)
    X_test, y_test = _load_data(df.iloc[ntrn:],30)

    return (X_train, y_train), (X_test, y_test)

In [13]:
(X_train, y_train), (X_test, y_test) = train_test_split(df)

In [21]:
from keras.models import Sequential  
from keras.layers.core import Dense, Activation, Dropout 
from keras.layers.recurrent import LSTM


hidden_neurons = 300

model = Sequential()
model.add(LSTM(300, input_shape=(30, 5), return_sequences=False))       
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation("tanh"))

model.compile(loss="mean_squared_error", optimizer="Adam")  

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 300)               367200    
_________________________________________________________________
dropout_7 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 301       
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 367,501
Trainable params: 367,501
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(X_train, y_train, batch_size=100, epochs=10, validation_split=0.05)  

Train on 831 samples, validate on 44 samples
Epoch 1/10
831/831 [==============================] - 5s - loss: 1.0953 - val_loss: 2.4457
Epoch 2/10
831/831 [==============================] - 2s - loss: 0.7549 - val_loss: 2.0043
Epoch 3/10
831/831 [==============================] - 3s - loss: 0.6905 - val_loss: 1.9080
Epoch 4/10
831/831 [==============================] - 2s - loss: 0.6584 - val_loss: 1.8595
Epoch 5/10
831/831 [==============================] - 2s - loss: 0.6414 - val_loss: 1.8015
Epoch 6/10
831/831 [==============================] - 2s - loss: 0.6215 - val_loss: 1.7586
Epoch 7/10
831/831 [==============================] - 2s - loss: 0.6011 - val_loss: 1.7470
Epoch 8/10
831/831 [==============================] - 2s - loss: 0.5960 - val_loss: 1.7111
Epoch 9/10
831/831 [==============================] - 2s - loss: 0.5937 - val_loss: 1.7046
Epoch 10/10
831/831 [==============================] - 2s - loss: 0.5868 - val_loss: 1.6863


In [23]:
predicted = model.predict(X_test) 

In [40]:
outcome = pd.DataFrame()
outcome['actual'] = y_test
outcome['predicted'] = predicted
outcome.to_csv("outcome.csv")